The Deep Speech pretrained model used for this project is the Deep Speech 0.9.3,which was last updated by Reuben Morais in December 2020.
(Morais, 2020) 
Available at http:deepspeech.readthedocs.io/en/v0.9.3/index.html 

In [3]:
! pip install deepspeech

     |████████████████████████████████| 9.2 MB 5.1 MB/s 


In [4]:
## Load packages

import os
import glob as gb
import numpy as np
import pandas as pd
from deepspeech import Model
import matplotlib.pyplot as plt
from scipy.io.wavfile import read

In [5]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  180M  100  180M    0     0  82.8M      0  0:00:02  0:00:02 --:--:--  107M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  909M  100  909M    0     0  39.3M      0  0:00:23  0:00:23 --:--:-- 34.9M


In [6]:
model_file_path = 'deepspeech-0.9.3-models.pbmm'
lm_file_path = 'deepspeech-0.9.3-models.scorer'
beam_width = 100
lm_alpha = 0.93
lm_beta = 1.18

model = Model(model_file_path)                    #Adding Acoustic Model
model.enableExternalScorer(lm_file_path)          #Activate Language Model

In [7]:
model.setScorerAlphaBeta(lm_alpha, lm_beta)
model.setBeamWidth(beam_width)

0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
## Read file

audio_link = '/content/drive/MyDrive/20 audio files/Selected/'
data = gb.glob(audio_link + '/*.wav')

In [ ]:
len(data)

20

In [9]:
audio_data = [read(audio)[1] for audio in data]
filenames = [os.path.basename(audio).split('.')[0] for audio in data]

In [10]:
### Import mets data

metadata = pd.read_csv('/content/drive/MyDrive/LJSpeech-1.1/metadata.csv', sep = '|', header=None, names=['ID', 'Transcription', 'Normalized_transcription'])
metadata.head()

,ID,Transcription,Normalized_transcription
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [11]:
df = pd.DataFrame(
    {
        'ID': filenames,
        'audio': audio_data
        
    }
)

df.head()

,ID,audio
0,LJ001-0025,"[-74, -95, -197, -214, -400, -399, -425, -432,..."
1,LJ001-0035,"[27, 23, 30, 26, 24, 27, 19, 26, 25, 16, 19, 1..."
2,LJ001-0056,"[9, 12, 10, 6, -4, -8, 0, 7, 13, 12, 10, 7, -3..."
3,LJ001-0062,"[-1, -4, -1, -2, 0, 0, 1, 3, 0, 2, 2, 1, 0, 2,..."
4,LJ001-0058,"[-4, -3, -1, 1, 0, 5, 9, 8, 5, 6, 13, 17, 16, ..."


In [ ]:
transcribe = lambda x: model.stt(x)
df.loc[:,'prediction'] = df.audio.apply(transcribe)

In [ ]:
df.drop(columns=['audio'], inplace=True)
df

,ID,prediction
0,LJ001-0025,it mertoun freer hand simpler rounder and less...
1,LJ001-0035,discarded this for a more completely roman and...
2,LJ001-0056,as these great venetian printers together with...
3,LJ001-0062,even in italy most of the pelagica and labors ...
4,LJ001-0058,worthy representatives of the eager enthusiasm...
5,LJ001-0071,to cartoons or even all your character beaucou...
6,LJ001-0085,reserved for the founders of the later eightee...
7,LJ001-0101,is discouraging to know that the improvement o...
8,LJ001-0099,must be remembered however that most modern pr...
9,LJ001-0109,is best for the bite avoidance of irrational s...


In [ ]:
new_df = pd.merge(df, metadata, on = 'ID')
new_df.head(20)

,ID,prediction,Transcription,Normalized_transcription
0,LJ001-0025,it mertoun freer hand simpler rounder and less...,"imitates a much freer hand, simpler, rounder, ...","imitates a much freer hand, simpler, rounder, ..."
1,LJ001-0035,discarded this for a more completely roman and...,they discarded this for a more completely Roma...,they discarded this for a more completely Roma...
2,LJ001-0056,as these great venetian printers together with...,"It was these great Venetian printers, together...","It was these great Venetian printers, together..."
3,LJ001-0062,even in italy most of the pelagica and labors ...,Even in Italy most of the theological and law ...,Even in Italy most of the theological and law ...
4,LJ001-0058,worthy representatives of the eager enthusiasm...,and are worthy representatives of the eager en...,and are worthy representatives of the eager en...
5,LJ001-0071,to cartoons or even all your character beaucou...,Most of Caxton's own types are of an earlier c...,Most of Caxton's own types are of an earlier c...
6,LJ001-0085,reserved for the founders of the later eightee...,It was reserved for the founders of the later ...,It was reserved for the founders of the later ...
7,LJ001-0101,is discouraging to know that the improvement o...,It is discouraging to note that the improvemen...,It is discouraging to note that the improvemen...
8,LJ001-0099,must be remembered however that most modern pr...,"It must be remembered, however, that most mode...","It must be remembered, however, that most mode..."
9,LJ001-0109,is best for the bite avoidance of irrational s...,this is best furthered by the avoidance of irr...,this is best furthered by the avoidance of irr...
